In [108]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

import plotly.express as px
color_scheme = px.colors.qualitative.T10

In [3]:
df = pd.read_csv('pokemon_alopez247.csv')
pd.set_option('display.max_columns', None)
df.head()

,Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,isLegendary,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Green,True,0.875,Monster,Grass,False,0.71,6.9,45,quadruped
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Green,True,0.875,Monster,Grass,False,0.99,13.0,45,quadruped
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Green,True,0.875,Monster,Grass,True,2.01,100.0,45,quadruped
3,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,Red,True,0.875,Monster,Dragon,False,0.61,8.5,45,bipedal_tailed
4,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,Red,True,0.875,Monster,Dragon,False,1.09,19.0,45,bipedal_tailed


In [114]:
type1 = df['Type_1'].value_counts()
fig = px.bar(x=type1.index, y=type1.values, color = type1.index, 
             color_discrete_sequence=color_scheme,text = type1.values, title='Type1')

fig.update_layout(
     xaxis_title="Type 1 ",
    yaxis_title="count",
    template = 'simple_white')
fig.show()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Number            721 non-null    int64  
 1   Name              721 non-null    object 
 2   Type_1            721 non-null    object 
 3   Type_2            350 non-null    object 
 4   Total             721 non-null    int64  
 5   HP                721 non-null    int64  
 6   Attack            721 non-null    int64  
 7   Defense           721 non-null    int64  
 8   Sp_Atk            721 non-null    int64  
 9   Sp_Def            721 non-null    int64  
 10  Speed             721 non-null    int64  
 11  Generation        721 non-null    int64  
 12  isLegendary       721 non-null    bool   
 13  Color             721 non-null    object 
 14  hasGender         721 non-null    bool   
 15  Pr_Male           644 non-null    float64
 16  Egg_Group_1       721 non-null    object 
 1

In [11]:
df['Pr_Male'].fillna(0,inplace=True)

In [12]:
df['Color'].value_counts()

Blue      134
Brown     110
Green      79
Red        75
Grey       69
Purple     65
Yellow     64
White      52
Pink       41
Black      32
Name: Color, dtype: int64

In [14]:
df['Body_Style'].value_counts()

bipedal_tailed      158
quadruped           135
bipedal_tailless    109
two_wings            63
head_arms            39
head_only            34
with_fins            31
insectoid            30
head_base            30
serpentine_body      29
four_wings           18
head_legs            17
multiple_bodies      15
several_limbs        13
Name: Body_Style, dtype: int64

In [15]:
df.drop(['Number','Name'],axis=1, inplace=True)

In [17]:
df.head()

,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,isLegendary,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
0,Grass,Poison,318,45,49,49,65,65,45,1,False,Green,True,0.875,Monster,Grass,False,0.71,6.9,45,quadruped
1,Grass,Poison,405,60,62,63,80,80,60,1,False,Green,True,0.875,Monster,Grass,False,0.99,13.0,45,quadruped
2,Grass,Poison,525,80,82,83,100,100,80,1,False,Green,True,0.875,Monster,Grass,True,2.01,100.0,45,quadruped
3,Fire,NaN,309,39,52,43,60,50,65,1,False,Red,True,0.875,Monster,Dragon,False,0.61,8.5,45,bipedal_tailed
4,Fire,NaN,405,58,64,58,80,65,80,1,False,Red,True,0.875,Monster,Dragon,False,1.09,19.0,45,bipedal_tailed


In [20]:
df[['hasGender','hasMegaEvolution']] = df[['hasGender','hasMegaEvolution']] * 1

In [21]:
df.head()

,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,isLegendary,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
0,Grass,Poison,318,45,49,49,65,65,45,1,False,Green,1,0.875,Monster,Grass,0,0.71,6.9,45,quadruped
1,Grass,Poison,405,60,62,63,80,80,60,1,False,Green,1,0.875,Monster,Grass,0,0.99,13.0,45,quadruped
2,Grass,Poison,525,80,82,83,100,100,80,1,False,Green,1,0.875,Monster,Grass,1,2.01,100.0,45,quadruped
3,Fire,NaN,309,39,52,43,60,50,65,1,False,Red,1,0.875,Monster,Dragon,0,0.61,8.5,45,bipedal_tailed
4,Fire,NaN,405,58,64,58,80,65,80,1,False,Red,1,0.875,Monster,Dragon,0,1.09,19.0,45,bipedal_tailed


In [30]:
cat_df = df.select_dtypes(object)

In [34]:
cat_dummies = pd.get_dummies(cat_df)

In [35]:
ndf = df.copy()

In [40]:
ndf.drop(cat_df.columns.to_list(),axis=1,inplace=True)

In [48]:
total = pd.concat([ndf,cat_dummies],axis=1)

In [52]:
total.head()

,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,isLegendary,hasGender,Pr_Male,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Type_1_Bug,Type_1_Dark,Type_1_Dragon,Type_1_Electric,Type_1_Fairy,Type_1_Fighting,Type_1_Fire,Type_1_Flying,Type_1_Ghost,Type_1_Grass,Type_1_Ground,Type_1_Ice,Type_1_Normal,Type_1_Poison,Type_1_Psychic,Type_1_Rock,Type_1_Steel,Type_1_Water,Type_2_Bug,Type_2_Dark,Type_2_Dragon,Type_2_Electric,Type_2_Fairy,Type_2_Fighting,Type_2_Fire,Type_2_Flying,Type_2_Ghost,Type_2_Grass,Type_2_Ground,Type_2_Ice,Type_2_Normal,Type_2_Poison,Type_2_Psychic,Type_2_Rock,Type_2_Steel,Type_2_Water,Color_Black,Color_Blue,Color_Brown,Color_Green,Color_Grey,Color_Pink,Color_Purple,Color_Red,Color_White,Color_Yellow,Egg_Group_1_Amorphous,Egg_Group_1_Bug,Egg_Group_1_Ditto,Egg_Group_1_Dragon,Egg_Group_1_Fairy,Egg_Group_1_Field,Egg_Group_1_Flying,Egg_Group_1_Grass,Egg_Group_1_Human-Like,Egg_Group_1_Mineral,Egg_Group_1_Monster,Egg_Group_1_Undiscovered,Egg_Group_1_Water_1,Egg_Group_1_Water_2,Egg_Group_1_Water_3,Egg_Group_2_Amorphous,Egg_Group_2_Bug,Egg_Group_2_Dragon,Egg_Group_2_Fairy,Egg_Group_2_Field,Egg_Group_2_Flying,Egg_Group_2_Grass,Egg_Group_2_Human-Like,Egg_Group_2_Mineral,Egg_Group_2_Monster,Egg_Group_2_Water_1,Egg_Group_2_Water_2,Egg_Group_2_Water_3,Body_Style_bipedal_tailed,Body_Style_bipedal_tailless,Body_Style_four_wings,Body_Style_head_arms,Body_Style_head_base,Body_Style_head_legs,Body_Style_head_only,Body_Style_insectoid,Body_Style_multiple_bodies,Body_Style_quadruped,Body_Style_serpentine_body,Body_Style_several_limbs,Body_Style_two_wings,Body_Style_with_fins
0,318,45,49,49,65,65,45,1,False,1,0.875,0,0.71,6.9,45,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,405,60,62,63,80,80,60,1,False,1,0.875,0,0.99,13.0,45,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,525,80,82,83,100,100,80,1,False,1,0.875,1,2.01,100.0,45,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,309,39,52,43,60,50,65,1,False,1,0.875,0,0.61,8.5,45,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,405,58,64,58,80,65,80,1,False,1,0.875,0,1.09,19.0,45,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
feature = total.drop('isLegendary',axis=1)
target = total['isLegendary']*1 

In [65]:
feature.head()

,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,hasGender,Pr_Male,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Type_1_Bug,Type_1_Dark,Type_1_Dragon,Type_1_Electric,Type_1_Fairy,Type_1_Fighting,Type_1_Fire,Type_1_Flying,Type_1_Ghost,Type_1_Grass,Type_1_Ground,Type_1_Ice,Type_1_Normal,Type_1_Poison,Type_1_Psychic,Type_1_Rock,Type_1_Steel,Type_1_Water,Type_2_Bug,Type_2_Dark,Type_2_Dragon,Type_2_Electric,Type_2_Fairy,Type_2_Fighting,Type_2_Fire,Type_2_Flying,Type_2_Ghost,Type_2_Grass,Type_2_Ground,Type_2_Ice,Type_2_Normal,Type_2_Poison,Type_2_Psychic,Type_2_Rock,Type_2_Steel,Type_2_Water,Color_Black,Color_Blue,Color_Brown,Color_Green,Color_Grey,Color_Pink,Color_Purple,Color_Red,Color_White,Color_Yellow,Egg_Group_1_Amorphous,Egg_Group_1_Bug,Egg_Group_1_Ditto,Egg_Group_1_Dragon,Egg_Group_1_Fairy,Egg_Group_1_Field,Egg_Group_1_Flying,Egg_Group_1_Grass,Egg_Group_1_Human-Like,Egg_Group_1_Mineral,Egg_Group_1_Monster,Egg_Group_1_Undiscovered,Egg_Group_1_Water_1,Egg_Group_1_Water_2,Egg_Group_1_Water_3,Egg_Group_2_Amorphous,Egg_Group_2_Bug,Egg_Group_2_Dragon,Egg_Group_2_Fairy,Egg_Group_2_Field,Egg_Group_2_Flying,Egg_Group_2_Grass,Egg_Group_2_Human-Like,Egg_Group_2_Mineral,Egg_Group_2_Monster,Egg_Group_2_Water_1,Egg_Group_2_Water_2,Egg_Group_2_Water_3,Body_Style_bipedal_tailed,Body_Style_bipedal_tailless,Body_Style_four_wings,Body_Style_head_arms,Body_Style_head_base,Body_Style_head_legs,Body_Style_head_only,Body_Style_insectoid,Body_Style_multiple_bodies,Body_Style_quadruped,Body_Style_serpentine_body,Body_Style_several_limbs,Body_Style_two_wings,Body_Style_with_fins
0,318,45,49,49,65,65,45,1,1,0.875,0,0.71,6.9,45,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,405,60,62,63,80,80,60,1,1,0.875,0,0.99,13.0,45,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,525,80,82,83,100,100,80,1,1,0.875,1,2.01,100.0,45,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,309,39,52,43,60,50,65,1,1,0.875,0,0.61,8.5,45,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,405,58,64,58,80,65,80,1,1,0.875,0,1.09,19.0,45,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
X_train , X_test , y_train , y_test = train_test_split(feature,target,test_size=0.2,random_state=123)

In [67]:
scaller = MinMaxScaler(feature_range=(0,1))
X_scalled = pd.DataFrame(data=scaller.fit_transform(X_train),columns=X_train.columns)
Xt_scalled = pd.DataFrame(data=scaller.transform(X_test),columns=X_test.columns)

In [74]:
os = SMOTE()
X_TRAIN_BALANCE , Y_TRAIN_BALANCE = os.fit_resample(X_scalled,y_train)
X_TEST_BALANCE , Y_TEST_BALANCE = os.fit_resample(Xt_scalled,y_test)

In [77]:
lg = LogisticRegression()
lg.fit(X_TRAIN_BALANCE,Y_TRAIN_BALANCE)
prediction = lg.predict(X_TEST_BALANCE)
F1 = f1_score(Y_TEST_BALANCE,prediction)
ACCURACY = accuracy_score(Y_TEST_BALANCE,prediction)
PRECISION = precision_score(Y_TEST_BALANCE,prediction)
RECALL = recall_score(Y_TEST_BALANCE,prediction)
m1 = pd.DataFrame({"F1SCORE":F1 , 'PRECISION':PRECISION, 'RECALL':RECALL , 'ACCURACY':ACCURACY},index=['LOGISTIC_REGRESSIO'])
m1

,F1SCORE,PRECISION,RECALL,ACCURACY
LOGISTIC_REGRESSIO,0.98917,0.978571,1.0,0.989051


In [78]:
tree = DecisionTreeClassifier()
tree.fit(X_TRAIN_BALANCE,Y_TRAIN_BALANCE)
prediction = tree.predict(X_TEST_BALANCE)
F1 = f1_score(Y_TEST_BALANCE,prediction)
ACCURACY = accuracy_score(Y_TEST_BALANCE,prediction)
PRECISION = precision_score(Y_TEST_BALANCE,prediction)
RECALL = recall_score(Y_TEST_BALANCE,prediction)
m2 = pd.DataFrame({"F1SCORE":F1 , 'PRECISION':PRECISION, 'RECALL':RECALL , 'ACCURACY':ACCURACY},index=['DECISION_TREE'])
m2

,F1SCORE,PRECISION,RECALL,ACCURACY
DECISION_TREE,0.97037,0.984962,0.956204,0.970803


In [79]:
forest = RandomForestClassifier()
forest.fit(X_TRAIN_BALANCE,Y_TRAIN_BALANCE)
prediction = forest.predict(X_TEST_BALANCE)
F1 = f1_score(Y_TEST_BALANCE,prediction)
ACCURACY = accuracy_score(Y_TEST_BALANCE,prediction)
PRECISION = precision_score(Y_TEST_BALANCE,prediction)
RECALL = recall_score(Y_TEST_BALANCE,prediction)
m3 = pd.DataFrame({"F1SCORE":F1 , 'PRECISION':PRECISION, 'RECALL':RECALL , 'ACCURACY':ACCURACY},index=['RANDOM_FOREST'])
m3

,F1SCORE,PRECISION,RECALL,ACCURACY
RANDOM_FOREST,0.992754,0.985612,1.0,0.992701


In [80]:
xgb = XGBClassifier()
xgb.fit(X_TRAIN_BALANCE,Y_TRAIN_BALANCE)
prediction = xgb.predict(X_TEST_BALANCE)
F1 = f1_score(Y_TEST_BALANCE,prediction)
ACCURACY = accuracy_score(Y_TEST_BALANCE,prediction)
PRECISION = precision_score(Y_TEST_BALANCE,prediction)
RECALL = recall_score(Y_TEST_BALANCE,prediction)
m4 = pd.DataFrame({"F1SCORE":F1 , 'PRECISION':PRECISION, 'RECALL':RECALL , 'ACCURACY':ACCURACY},index=['XGB_REGRESSOR'])
m4

,F1SCORE,PRECISION,RECALL,ACCURACY
XGB_REGRESSOR,0.992754,0.985612,1.0,0.992701


In [81]:
knn = KNeighborsClassifier()
knn.fit(X_TRAIN_BALANCE,Y_TRAIN_BALANCE)
prediction = knn.predict(X_TEST_BALANCE)
F1 = f1_score(Y_TEST_BALANCE,prediction)
ACCURACY = accuracy_score(Y_TEST_BALANCE,prediction)
PRECISION = precision_score(Y_TEST_BALANCE,prediction)
RECALL = recall_score(Y_TEST_BALANCE,prediction)
m5 = pd.DataFrame({"F1SCORE":F1 , 'PRECISION':PRECISION, 'RECALL':RECALL , 'ACCURACY':ACCURACY},index=['Kneighbors'])
m5

,F1SCORE,PRECISION,RECALL,ACCURACY
Kneighbors,0.98917,0.978571,1.0,0.989051


In [82]:
sv = SVC()
sv.fit(X_TRAIN_BALANCE,Y_TRAIN_BALANCE)
prediction = sv.predict(X_TEST_BALANCE)
F1 = f1_score(Y_TEST_BALANCE,prediction)
ACCURACY = accuracy_score(Y_TEST_BALANCE,prediction)
PRECISION = precision_score(Y_TEST_BALANCE,prediction)
RECALL = recall_score(Y_TEST_BALANCE,prediction)
m6 = pd.DataFrame({"F1SCORE":F1 , 'PRECISION':PRECISION, 'RECALL':RECALL , 'ACCURACY':ACCURACY},index=['SVC'])
m6

,F1SCORE,PRECISION,RECALL,ACCURACY
SVC,0.985401,0.985401,0.985401,0.985401


In [99]:
ct = CatBoostClassifier()
ct.fit(X_TRAIN_BALANCE,Y_TRAIN_BALANCE)
prediction = ct.predict(X_TEST_BALANCE)
F1 = f1_score(Y_TEST_BALANCE,prediction)
ACCURACY = accuracy_score(Y_TEST_BALANCE,prediction)
PRECISION = precision_score(Y_TEST_BALANCE,prediction)
RECALL = recall_score(Y_TEST_BALANCE,prediction)
m7 = pd.DataFrame({"F1SCORE":F1 , 'PRECISION':PRECISION, 'RECALL':RECALL , 'ACCURACY':ACCURACY},index=['CATBOOST_CLASSIFIER'])
m7

Learning rate set to 0.01063
0:	learn: 0.6702529	total: 144ms	remaining: 2m 23s
1:	learn: 0.6477362	total: 149ms	remaining: 1m 14s
2:	learn: 0.6253288	total: 151ms	remaining: 50s
3:	learn: 0.6031914	total: 156ms	remaining: 38.8s
4:	learn: 0.5796605	total: 161ms	remaining: 32s
5:	learn: 0.5570311	total: 167ms	remaining: 27.7s
6:	learn: 0.5393807	total: 174ms	remaining: 24.7s
7:	learn: 0.5181232	total: 180ms	remaining: 22.3s
8:	learn: 0.4984080	total: 188ms	remaining: 20.7s
9:	learn: 0.4797793	total: 195ms	remaining: 19.3s
10:	learn: 0.4616858	total: 200ms	remaining: 18s
11:	learn: 0.4445916	total: 206ms	remaining: 17s
12:	learn: 0.4287070	total: 211ms	remaining: 16s
13:	learn: 0.4130698	total: 217ms	remaining: 15.3s
14:	learn: 0.3994028	total: 223ms	remaining: 14.7s
15:	learn: 0.3854538	total: 229ms	remaining: 14.1s
16:	learn: 0.3728388	total: 236ms	remaining: 13.7s
17:	learn: 0.3602113	total: 242ms	remaining: 13.2s
18:	learn: 0.3475091	total: 248ms	remaining: 12.8s
19:	learn: 0.3348210

161:	learn: 0.0153545	total: 1.07s	remaining: 5.54s
162:	learn: 0.0153372	total: 1.07s	remaining: 5.52s
163:	learn: 0.0152748	total: 1.08s	remaining: 5.51s
164:	learn: 0.0151135	total: 1.09s	remaining: 5.51s
165:	learn: 0.0150867	total: 1.09s	remaining: 5.5s
166:	learn: 0.0149912	total: 1.1s	remaining: 5.48s
167:	learn: 0.0149585	total: 1.1s	remaining: 5.47s
168:	learn: 0.0148134	total: 1.11s	remaining: 5.46s
169:	learn: 0.0147060	total: 1.11s	remaining: 5.44s
170:	learn: 0.0145772	total: 1.12s	remaining: 5.43s
171:	learn: 0.0144147	total: 1.12s	remaining: 5.41s
172:	learn: 0.0142840	total: 1.13s	remaining: 5.4s
173:	learn: 0.0141159	total: 1.14s	remaining: 5.39s
174:	learn: 0.0139660	total: 1.14s	remaining: 5.39s
175:	learn: 0.0138581	total: 1.15s	remaining: 5.38s
176:	learn: 0.0136849	total: 1.16s	remaining: 5.37s
177:	learn: 0.0135175	total: 1.16s	remaining: 5.36s
178:	learn: 0.0134558	total: 1.17s	remaining: 5.35s
179:	learn: 0.0133056	total: 1.17s	remaining: 5.34s
180:	learn: 0.01

348:	learn: 0.0057775	total: 2.15s	remaining: 4s
349:	learn: 0.0057305	total: 2.15s	remaining: 4s
350:	learn: 0.0057227	total: 2.16s	remaining: 3.99s
351:	learn: 0.0056939	total: 2.16s	remaining: 3.98s
352:	learn: 0.0056792	total: 2.17s	remaining: 3.98s
353:	learn: 0.0056648	total: 2.18s	remaining: 3.97s
354:	learn: 0.0056340	total: 2.18s	remaining: 3.96s
355:	learn: 0.0056263	total: 2.19s	remaining: 3.96s
356:	learn: 0.0056208	total: 2.19s	remaining: 3.95s
357:	learn: 0.0056061	total: 2.2s	remaining: 3.94s
358:	learn: 0.0055991	total: 2.21s	remaining: 3.94s
359:	learn: 0.0055959	total: 2.21s	remaining: 3.93s
360:	learn: 0.0055755	total: 2.22s	remaining: 3.92s
361:	learn: 0.0055671	total: 2.22s	remaining: 3.92s
362:	learn: 0.0055334	total: 2.23s	remaining: 3.91s
363:	learn: 0.0055181	total: 2.23s	remaining: 3.9s
364:	learn: 0.0054882	total: 2.24s	remaining: 3.9s
365:	learn: 0.0054839	total: 2.25s	remaining: 3.89s
366:	learn: 0.0054593	total: 2.25s	remaining: 3.88s
367:	learn: 0.0054353

535:	learn: 0.0033360	total: 3.22s	remaining: 2.79s
536:	learn: 0.0033176	total: 3.23s	remaining: 2.79s
537:	learn: 0.0033147	total: 3.24s	remaining: 2.78s
538:	learn: 0.0033045	total: 3.24s	remaining: 2.77s
539:	learn: 0.0033029	total: 3.25s	remaining: 2.77s
540:	learn: 0.0032998	total: 3.25s	remaining: 2.76s
541:	learn: 0.0032978	total: 3.26s	remaining: 2.76s
542:	learn: 0.0032794	total: 3.27s	remaining: 2.75s
543:	learn: 0.0032728	total: 3.27s	remaining: 2.74s
544:	learn: 0.0032704	total: 3.28s	remaining: 2.74s
545:	learn: 0.0032687	total: 3.29s	remaining: 2.73s
546:	learn: 0.0032628	total: 3.29s	remaining: 2.73s
547:	learn: 0.0032467	total: 3.3s	remaining: 2.72s
548:	learn: 0.0032370	total: 3.31s	remaining: 2.71s
549:	learn: 0.0032253	total: 3.31s	remaining: 2.71s
550:	learn: 0.0032112	total: 3.32s	remaining: 2.71s
551:	learn: 0.0032082	total: 3.33s	remaining: 2.7s
552:	learn: 0.0032082	total: 3.33s	remaining: 2.69s
553:	learn: 0.0031939	total: 3.34s	remaining: 2.69s
554:	learn: 0.

704:	learn: 0.0024692	total: 4.31s	remaining: 1.8s
705:	learn: 0.0024621	total: 4.31s	remaining: 1.8s
706:	learn: 0.0024621	total: 4.32s	remaining: 1.79s
707:	learn: 0.0024622	total: 4.33s	remaining: 1.78s
708:	learn: 0.0024622	total: 4.33s	remaining: 1.78s
709:	learn: 0.0024597	total: 4.34s	remaining: 1.77s
710:	learn: 0.0024597	total: 4.34s	remaining: 1.76s
711:	learn: 0.0024597	total: 4.35s	remaining: 1.76s
712:	learn: 0.0024482	total: 4.36s	remaining: 1.75s
713:	learn: 0.0024482	total: 4.36s	remaining: 1.75s
714:	learn: 0.0024482	total: 4.37s	remaining: 1.74s
715:	learn: 0.0024482	total: 4.37s	remaining: 1.74s
716:	learn: 0.0024482	total: 4.38s	remaining: 1.73s
717:	learn: 0.0024482	total: 4.38s	remaining: 1.72s
718:	learn: 0.0024458	total: 4.39s	remaining: 1.72s
719:	learn: 0.0024458	total: 4.4s	remaining: 1.71s
720:	learn: 0.0024458	total: 4.41s	remaining: 1.7s
721:	learn: 0.0024440	total: 4.41s	remaining: 1.7s
722:	learn: 0.0024332	total: 4.42s	remaining: 1.69s
723:	learn: 0.002

885:	learn: 0.0022837	total: 5.38s	remaining: 692ms
886:	learn: 0.0022837	total: 5.38s	remaining: 686ms
887:	learn: 0.0022837	total: 5.39s	remaining: 680ms
888:	learn: 0.0022837	total: 5.4s	remaining: 674ms
889:	learn: 0.0022837	total: 5.4s	remaining: 668ms
890:	learn: 0.0022837	total: 5.41s	remaining: 662ms
891:	learn: 0.0022837	total: 5.41s	remaining: 656ms
892:	learn: 0.0022837	total: 5.42s	remaining: 649ms
893:	learn: 0.0022837	total: 5.42s	remaining: 643ms
894:	learn: 0.0022815	total: 5.43s	remaining: 637ms
895:	learn: 0.0022815	total: 5.43s	remaining: 631ms
896:	learn: 0.0022815	total: 5.44s	remaining: 625ms
897:	learn: 0.0022815	total: 5.45s	remaining: 619ms
898:	learn: 0.0022815	total: 5.45s	remaining: 613ms
899:	learn: 0.0022815	total: 5.46s	remaining: 606ms
900:	learn: 0.0022815	total: 5.46s	remaining: 600ms
901:	learn: 0.0022815	total: 5.47s	remaining: 594ms
902:	learn: 0.0022815	total: 5.48s	remaining: 588ms
903:	learn: 0.0022815	total: 5.48s	remaining: 582ms
904:	learn: 0.

,F1SCORE,PRECISION,RECALL,ACCURACY
CATBOOST_CLASSIFIER,0.992754,0.985612,1.0,0.992701


In [100]:
result = pd.concat([m1,m2,m3,m4,m5,m6,m7],axis=0)
cm = sns.light_palette("green", as_cmap=True)
result.style.background_gradient(cmap=cm).set_precision(5)

C:\Users\IsoorI\AppData\Local\Temp\ipykernel_2784\280318686.py:3: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  result.style.background_gradient(cmap=cm).set_precision(5)


,F1SCORE,PRECISION,RECALL,ACCURACY
LOGISTIC_REGRESSIO,0.98917,0.97857,1.00000,0.98905
DECISION_TREE,0.97037,0.98496,0.95620,0.97080
RANDOM_FOREST,0.99275,0.98561,1.00000,0.99270
XGB_REGRESSOR,0.99275,0.98561,1.00000,0.99270
Kneighbors,0.98917,0.97857,1.00000,0.98905
SVC,0.98540,0.98540,0.98540,0.98540
CATBOOST_CLASSIFIER,0.99275,0.98561,1.00000,0.99270


In [103]:
prediction[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
      dtype=int64)

In [104]:
Y_TEST_BALANCE[:20]

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    1
16    0
17    1
18    0
19    0
Name: isLegendary, dtype: int32